<a href="https://colab.research.google.com/github/dorahs71/ml-course/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# “What sorts of people were more likely to survive?”

using passenger data (ie name, age, gender, socio-economic class, etc)

In [83]:
import urllib.request as req
url = "https://github.com/Elwing-Chou/ximen0624/raw/main/titanic/train.csv"
req.urlretrieve(url, "train.csv")

url = "https://github.com/Elwing-Chou/ximen0624/raw/main/titanic/test.csv"
req.urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7cb76fdfc7c0>)

In [84]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8") #永遠都用utf-8
test = pd.read_csv("test.csv", encoding="utf-8")

train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# 資料預處理
- 不能有空資料，要填缺失值
- 把不需要的欄位去掉：PassengerId, Survived
- 一次處理完 train & test
- 好好理解自己的資料，例如：車票號碼，可以反映有哪些人是一起買票的，他們可能是朋友、情人、其他親密關係
- Cabin 雖然很多沒有資料，但因為可以反映船上的人際和財務能力，還是納入考量(只先抓前綴字母)
- Embarked：上船的地點



In [85]:
datas = pd.concat([train, test], axis=0) #用 axis=0 來合併 item 數量
datas = datas.drop(["PassengerId", "Survived"], axis=1) #用 axis=1 來 drop 欄位

In [86]:
# 語法教學
# Everything is an object: 任何東西皆具有型態，每個型態具有其獨特操作
# 型態: dict, 操作： [key]
# 功能也是如此，型態：int -> 只是步驟, 操作: (4.2) -> 加小括號表示真的執行

# def test(mode):
#   if mode is True:
#     return int
#   else:
#     return round

# test(False)(10000.6) -> 10001
# test(True)(10000.6) -> 10000


In [87]:
# apply: 轉換
def func(a):
  return (a ** 2) * 10

s = pd.Series([1, 2, 3, 4])
s.apply(func) # = (s ** 2) * 10


0     10
1     40
2     90
3    160
dtype: int64

In [88]:
def func(s):
  return (s.split(".")[0]
          .split(",")[-1]
          .strip())
datas["Name"] = datas["Name"].apply(func)

datas["Name"]

0          Mr
1         Mrs
2        Miss
3         Mrs
4          Mr
        ...  
413        Mr
414      Dona
415        Mr
416        Mr
417    Master
Name: Name, Length: 1309, dtype: object

In [89]:
def func(s):
  if pd.isna(s):
    return s
  else:
    return s[0]

datas["Cabin"] = datas["Cabin"].apply(func)

datas["Cabin"]

0      NaN
1        C
2      NaN
3        C
4      NaN
      ... 
413    NaN
414      C
415    NaN
416    NaN
417    NaN
Name: Cabin, Length: 1309, dtype: object

In [90]:
# datas["Name"].value_counts()

In [91]:
counts = datas["Ticket"].value_counts()

def func(t):
  if pd.isna(t):
    return t
  else:
    return counts[t]

datas["Ticket"] = datas["Ticket"].apply(func)

datas["Ticket"]

0      1
1      2
2      1
3      2
4      1
      ..
413    1
414    3
415    1
416    1
417    3
Name: Ticket, Length: 1309, dtype: int64

# 欄位分別
1. 類別：固定幾種，（Pclass, Name, Sex, Cabin, Embarked）

2. 數值：連續型，(Age, SibSp, Parch Ticket#, Fare)

# 資料預處理
1. 填缺失值：
- a 類別：補最常出現的類別
- b 數值：補中位數，表示所有資料中有一半大於，一半小於


P.S 老師補充了「偏度」現在的算法，已經和以前的統計學概念不一樣了，以前會說，右偏的話，中位數小於平均數，但其實這兩個的數學意義不同，也沒有這個數學關係

# 要當個懷疑論者


In [92]:
datas.isna() #True 表空值

s = datas.isna().sum() #表該欄位是空的的總數

# 篩選 Series[跟你 Series 比數一樣多的 True/False]

fil = s > 0

s[fil]

Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [93]:
# 類別型：補最常出現的

most = datas["Embarked"].value_counts().idxmax() #S:914 最多，取最大值 index

datas["Embarked"] = datas["Embarked"].fillna(most)

In [94]:
# 數值型：補中位數

s = datas.dtypes
fil = s != "object"
num_index = s[fil].drop("Pclass").index
med = datas[num_index].median()

datas = datas.fillna(med)

# Pclass 雖然資料型別是數字，但以現時邏輯來看，算是 “類別” 喔！如果他有缺，應該要補成最常出現的

In [95]:
datas.isna() #True 表空值

s = datas.isna().sum() #表該欄位是空的的總數

# 篩選 Series[跟你 Series 比數一樣多的 True/False]

fil = s > 0

s[fil]

Cabin    1014
dtype: int64